In [ ]:
import numpy as np 

In [1]:
def initialiser_parametres(input,output):
    W=np.random.randn(input,output)*0.01
    b=np.zeros((1,output))
    return W,b

In [2]:
#propagation 
def propagation(X,W,b):
    Z=np.dot(X,W)+b
    return Z

In [3]:
#erreur quadratique moyenne
def erreur_quadratique(y_pred,y_true):
    return np.mean(np.square(y_pred-y_true))

In [4]:
def erreur_sortie(y_true,y_pred):
    return y_pred*(1-y_pred)*(y_true-y_pred)

In [20]:
#retropropagation
def retropropagation(X,y_true,y_pred):
    delta_sortie=erreur_sortie(y_true,y_pred)
    dW=np.dot(X.T,delta_sortie)
    db=np.sum(delta_sortie,axis=0,keepdims=True)
    return dW, db

In [21]:
#mise a jour parametres
def update_parametres(W,b,dW,db,taux_appr):
    W+=taux_appr*dW
    b+=taux_appr*db
    return W,b

In [22]:
#entrainement du modèle
def train(X_train, y_train,input,output,epochs,taux_appr):
    W,b=initialiser_parametres(input,output)

    for epoch in range(epochs):
        #propagation avant
        Z=propagation(X_train,W,b)
        #calcul erreur
        loss=erreur_quadratique(Z,y_train)
        if epoch % 100 ==0:
            print(f"-------Epoch{epoch+1}/{epochs}, Loss{loss}-------")

        #retropropagation
        dW,db=retropropagation(X_train,y_train,Z)
        #mise a jour des parametres
        W,b=update_parametres(W,b,dW,db,taux_appr)


    return W,b

In [23]:
#prediction
def predict(X,W,b):
    Z=propagation(X,W,b)
    return np.argmax(Z,axis=1)

In [24]:
#one hot encoding les sorties avec le nombre des classes:
def one_hot_encode(y, num_classes):
    one_hot = np.zeros((y.size, num_classes))
    one_hot[np.arange(y.size), y] = 1
    return one_hot



In [25]:
#echantillon:
#on a par exemple pour la classe 1: 3,6,9,12,15 sont activés
#pour classe 2: 1,2,3,6,9,8,7,10,13,14,15 sont activés 
#pour classe 3: 1,2,3,6,7,8,9,12,13,14,15 sont activés
#classe 4: 1,3,4,6,7,8,9,12,15
#classe5: 1,2,3,4,5,8,9,12,13,14,15
#classe 6 : 1,2,3,4,7,8,9,10,12,13,14,15
#classe 7: 1,2,3,6,7,8,9,12,15
#classe 8 : 1,2,3,4,6,7,8,9,10,12,13,14,15
#classe9: 1,2,3,4,6,7,8,9,12,13,14,15
import numpy as np

activation_patterns = {
    1: [3, 6, 9, 12, 15],
    2: [1, 2, 3, 6, 7, 8, 9, 10, 13, 14, 15],
    3: [1, 2, 3, 6, 7, 8, 9, 12, 13, 14, 15],
    4: [1, 3, 4, 6, 7, 8, 9, 12, 15],
    5: [1, 2, 3, 4, 5, 8, 9, 12, 13, 14, 15],
    6: [1, 2, 3, 4, 7, 8, 9, 10, 12, 13, 14, 15],
    7: [1, 2, 3, 6, 7, 8, 9, 12, 15],
    8: [1, 2, 3, 4, 6, 7, 8, 9, 10, 12, 13, 14, 15],
    9: [1, 2, 3, 4, 6, 7, 8, 9, 12, 13, 14, 15],
}

n_samples = 1000

X_train = np.zeros((n_samples, 15), dtype=int)  #les pixels
y_train = np.zeros(n_samples, dtype=int)    #les classes

#generer donnés
for i in range(n_samples):
    label = np.random.randint(1, 10)
    y_train[i] = label
    
    activation_indices = activation_patterns[label]
    
    X_train[i, np.array(activation_indices) - 1] = 1  # Soustraction de 1 pour l'indexation 0-basée

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

print("\nExemple d'X_train:", X_train[0])
print("Classe correspondante (y_train):", y_train[0])


X_train shape: (1000, 15)
y_train shape: (1000,)

Exemple d'X_train: [1 1 1 1 0 1 1 1 1 1 0 1 1 1 1]
Classe correspondante (y_train): 8


In [29]:
y_train_one_hot=one_hot_encode(y_train,10)
#hyperparams

input_size=15
output_size=10
epochs=1000
taux_appr=0.000001
#entrainement
W,b=train(X_train,y_train_one_hot,input_size,output_size,epochs,taux_appr)

-------Epoch1/1000, Loss0.101816075056823-------
-------Epoch101/1000, Loss0.1023256664119178-------
-------Epoch201/1000, Loss0.10304073964111382-------
-------Epoch301/1000, Loss0.10408061465832599-------
-------Epoch401/1000, Loss0.10567027693390182-------
-------Epoch501/1000, Loss0.10828994875271046-------
-------Epoch601/1000, Loss0.11317962657442672-------
-------Epoch701/1000, Loss0.12482765706291607-------
-------Epoch801/1000, Loss0.18305044315357136-------
-------Epoch901/1000, Lossnan-------


C:\Users\Anis\AppData\Local\Temp\ipykernel_9908\884405974.py:3: RuntimeWarning: overflow encountered in square
  return np.mean(np.square(y_pred-y_true))
C:\Users\Anis\AppData\Local\Temp\ipykernel_9908\3054478958.py:2: RuntimeWarning: overflow encountered in multiply
  return y_pred*(1-y_pred)*(y_true-y_pred)


## Propagation floue

In [ ]:
# -------------------------------------------Perceptron Floue --------------------------------------

In [34]:
def boltzmann_distribution(scores, T):
    """
    Calcule les probabilités selon la distribution de Boltzmann.
    scores : Les scores bruts du réseau (énergies associées aux classes)
    T : Température qui contrôle la stochastique de l'échantillonnage
    """
    exp_scores = np.exp(-scores / T)
    probabilities = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    return probabilities

def boltzmann_sampling(probabilities):
    """
    Echantillonne une classe en fonction des probabilités de Boltzmann.
    """
    # Échantillonner à partir des probabilités
    return np.array([np.random.choice(len(p), p=p) for p in probabilities])

In [35]:
def propagation_floue(X, W, b, T):
    """
    Propagation avant avec distribution de Boltzmann.
    X : Entrée
    W : Poids
    b : Biais
    T : Température pour la distribution de Boltzmann
    """
    Z = np.dot(X, W) + b  # Calcul des sorties brutes
    probabilities = boltzmann_distribution(Z, T)  # Calcul des probabilités selon Boltzmann
    y_pred = boltzmann_sampling(probabilities)    # Echantillonner les classes en fonction des probabilités
    return y_pred


In [36]:
#entrainement du modèle
def train_floue(X_train, y_train,input,output,epochs,taux_appr):
    W,b=initialiser_parametres(input,output)

    for epoch in range(epochs):
        #propagation avant
        Z=propagation_floue(X_train,W,b)
        #calcul erreur
        loss=erreur_quadratique(Z,y_train)
        if epoch % 100 ==0:
            print(f"-------Epoch{epoch+1}/{epochs}, Loss{loss}-------")

        #retropropagation
        dW,db=retropropagation(X_train,y_train,Z)
        #mise a jour des parametres
        W,b=update_parametres(W,b,dW,db,taux_appr)


    return W,b

In [37]:
y_train_one_hot=one_hot_encode(y_train,10)
#hyperparams

input_size=15
output_size=10
epochs=1000
taux_appr=0.000001
#entrainement
W,b=train(X_train,y_train_one_hot,input_size,output_size,epochs,taux_appr)

-------Epoch1/1000, Loss0.09936806225993473-------
-------Epoch101/1000, Loss0.09938958276979061-------
-------Epoch201/1000, Loss0.09943949622413671-------
-------Epoch301/1000, Loss0.09952625046705191-------
-------Epoch401/1000, Loss0.09966137342091771-------
-------Epoch501/1000, Loss0.09986078226432317-------
-------Epoch601/1000, Loss0.10014692224857565-------
-------Epoch701/1000, Loss0.10055242939281007-------
-------Epoch801/1000, Loss0.10112677963525114-------
-------Epoch901/1000, Loss0.10194927994036072-------
